<a href="https://colab.research.google.com/github/nohjuhyeon/study_data_analytics/blob/main/docs/quests/EDAs/EDA_ShoppingMallDeliveryWithDate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EDA 각 과정에서 insight 도출
- 제안 3: 배송 지연 요인 분석
  - 문제 정의: 배송 지연의 주요 요인을 식별하고 이에 대한 해결책을 도출합니다.
  - 배경: 배송 지연은 고객 불만의 주요 원인이 될 수 있으므로, 이를 최소화하기 위한 조치가 필요합니다.




In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
df_SMDW = pd.read_csv('/content/ShoppingMallDeliveryWithDate.csv')
df_SMDW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46749 entries, 0 to 46748
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   결제수단    46749 non-null  object 
 1   배송번호    46749 non-null  object 
 2   배송시작일   46749 non-null  object 
 3   배송완료일   46749 non-null  object 
 4   상품구매금액  46749 non-null  int64  
 5   상품번호    46661 non-null  float64
 6   수량      46749 non-null  int64  
 7   주문경로    46749 non-null  object 
 8   주문일     46749 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.2+ MB


In [2]:
df_SMDW.isnull().sum()

결제수단       0
배송번호       0
배송시작일      0
배송완료일      0
상품구매금액     0
상품번호      88
수량         0
주문경로       0
주문일        0
dtype: int64

In [3]:
df_SMDW[df_SMDW['상품번호'].isna()]
# 상품번호가 NaN인 데이터와 다른 column과의 연관성을 찾아봤을 때 확실한 결과를 찾기 어려움

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일
45143,카드,D-20170221-0000034-01,2017-02-22 오후 12:03,2017-02-23 오후 9:18,53000,NaN,1,네이버 페이,20170221
45152,"쿠폰,카드",D-20170302-0000152-00,2017-03-03 오후 4:12,2017-03-06 오후 4:47,7550,NaN,1,PC쇼핑몰,20170302
45178,무통장입금,D-20170325-0000011-00,2017-03-27 오후 5:06,2017-04-01 오후 7:48,15480,NaN,1,PC쇼핑몰,20170325
45194,"쿠폰,무통장입금",D-20170330-0000023-01,2017-03-31 오후 7:37,2017-04-01 오후 7:48,57600,NaN,1,PC쇼핑몰,20170330
45196,무통장입금,D-20170401-0000027-00,2017-04-03 오후 5:17,2017-04-10 오전 9:00,44000,NaN,1,PC쇼핑몰,20170401
...,...,...,...,...,...,...,...,...,...
45416,카드,D-20170605-0000056-00,2017-06-07 오후 4:19,2017-06-13 오전 5:42,12030,NaN,1,모바일웹,20170605
45417,카드,D-20170606-0000010-00,2017-06-07 오후 4:19,2017-06-13 오전 5:42,66500,NaN,1,PC쇼핑몰,20170606
45418,카드,D-20170606-0000026-01,2017-06-07 오후 4:21,2017-06-13 오전 11:58,12030,NaN,1,네이버 페이,20170606
45421,"적립금,무통장입금",D-20170607-0000018-00,2017-06-07 오후 4:19,2017-06-13 오전 5:42,64800,NaN,1,PC쇼핑몰,20170607


In [4]:
def convert_date(x):
    date, ampm, time = x.split()
    return pd.to_datetime(f"{date}", format="%Y-%m-%d")
def convert_time(x):
    date, ampm, time = x.split()
    if ampm == "오후" and time != "12:00":
        hour, minute = map(int, time.split(":"))
        time = f"{hour+12 if hour != 12 else hour}:{minute}"
    return pd.to_datetime(f"{date} {time}", format="%Y-%m-%d %H:%M")

In [5]:
df_SMDW['배송시작일_date'] = df_SMDW['배송시작일'].apply(convert_date)
df_SMDW['배송완료일_date'] = df_SMDW['배송완료일'].apply(convert_date)
df_SMDW['배송시작일_time'] = df_SMDW['배송시작일'].apply(convert_time)
df_SMDW['배송완료일_time'] = df_SMDW['배송완료일'].apply(convert_time)
df_SMDW['주문일_date']=pd.to_datetime(df_SMDW['주문일'],format="%Y%m%d")
df_SMDW

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송시작일_date,배송완료일_date,배송시작일_time,배송완료일_time,주문일_date
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077.0,1,PC쇼핑몰,20181227,2019-01-07,2019-01-11,2019-01-07 08:56:00,2019-01-11 05:32:00,2018-12-27
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42.0,2,모바일웹,20181229,2019-01-03,2019-01-07,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-29
2,무통장입금,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,1271.0,1,모바일웹,20181230,2019-01-03,2019-01-07,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-30
3,무통장입금,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,1085.0,1,모바일웹,20181231,2019-01-03,2019-01-07,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-31
4,무통장입금,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,1106.0,1,모바일웹,20181231,2019-01-07,2019-01-11,2019-01-07 08:56:00,2019-01-11 05:32:00,2018-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46744,카드,D-20171229-0000036-00,2017-12-29 오후 4:32,2018-01-02 오전 5:31,52800,445.0,1,모바일웹,20171229,2017-12-29,2018-01-02,2017-12-29 16:32:00,2018-01-02 05:31:00,2017-12-29
46745,카드,D-20171229-0000043-00,2018-01-03 오전 9:45,2018-01-07 오전 5:38,112200,172.0,2,모바일웹,20171229,2018-01-03,2018-01-07,2018-01-03 09:45:00,2018-01-07 05:38:00,2017-12-29
46746,카드,D-20171230-0000022-01,2018-01-02 오후 7:06,2018-01-11 오전 2:46,36000,140.0,1,네이버 페이,20171230,2018-01-02,2018-01-11,2018-01-02 19:06:00,2018-01-11 02:46:00,2017-12-30
46747,카드,D-20171231-0000017-01,2018-01-03 오후 2:00,2018-01-06 오후 6:06,73460,184.0,1,네이버 페이,20171231,2018-01-03,2018-01-06,2018-01-03 14:00:00,2018-01-06 18:06:00,2017-12-31


In [6]:
# 배송기간
df_SMDW["배송시작_배송완료"] = (df_SMDW['배송완료일_date'] - df_SMDW['배송시작일_date']).dt.days
df_SMDW["주문일_배송시작"] =  (df_SMDW['배송시작일_date'] - df_SMDW['주문일_date']).dt.days
df_SMDW["주문일_배송완료"] =  (df_SMDW['배송완료일_date'] - df_SMDW['주문일_date']).dt.days

df_SMDW

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송시작일_date,배송완료일_date,배송시작일_time,배송완료일_time,주문일_date,배송시작_배송완료,주문일_배송시작,주문일_배송완료
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077.0,1,PC쇼핑몰,20181227,2019-01-07,2019-01-11,2019-01-07 08:56:00,2019-01-11 05:32:00,2018-12-27,4,11,15
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42.0,2,모바일웹,20181229,2019-01-03,2019-01-07,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-29,4,5,9
2,무통장입금,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,1271.0,1,모바일웹,20181230,2019-01-03,2019-01-07,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-30,4,4,8
3,무통장입금,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,1085.0,1,모바일웹,20181231,2019-01-03,2019-01-07,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-31,4,3,7
4,무통장입금,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,1106.0,1,모바일웹,20181231,2019-01-07,2019-01-11,2019-01-07 08:56:00,2019-01-11 05:32:00,2018-12-31,4,7,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46744,카드,D-20171229-0000036-00,2017-12-29 오후 4:32,2018-01-02 오전 5:31,52800,445.0,1,모바일웹,20171229,2017-12-29,2018-01-02,2017-12-29 16:32:00,2018-01-02 05:31:00,2017-12-29,4,0,4
46745,카드,D-20171229-0000043-00,2018-01-03 오전 9:45,2018-01-07 오전 5:38,112200,172.0,2,모바일웹,20171229,2018-01-03,2018-01-07,2018-01-03 09:45:00,2018-01-07 05:38:00,2017-12-29,4,5,9
46746,카드,D-20171230-0000022-01,2018-01-02 오후 7:06,2018-01-11 오전 2:46,36000,140.0,1,네이버 페이,20171230,2018-01-02,2018-01-11,2018-01-02 19:06:00,2018-01-11 02:46:00,2017-12-30,9,3,12
46747,카드,D-20171231-0000017-01,2018-01-03 오후 2:00,2018-01-06 오후 6:06,73460,184.0,1,네이버 페이,20171231,2018-01-03,2018-01-06,2018-01-03 14:00:00,2018-01-06 18:06:00,2017-12-31,3,3,6


In [7]:
df_SMDW['수량'].value_counts()

1     41110
2      4199
3       878
4       252
5       223
6        40
10       16
7        11
8         7
15        5
17        2
16        1
46        1
20        1
11        1
50        1
12        1
Name: 수량, dtype: int64

In [8]:
# 수량 분류
bins = [0,1,5,10,10000]
labels = ['1','2-5','6-10','11 +']
df_SMDW['수량_category']=pd.cut(x=df_SMDW['수량'],bins=bins,labels=labels)
# df_data[:10]
df_SMDW['수량_category'].value_counts()

1       41110
2-5      5552
6-10       74
11 +       13
Name: 수량_category, dtype: int64

In [9]:
bins = [0,5,10,20,30,100000]
labels = ['0-5','6-10','11-20','21-30','31 +']
df_SMDW['배송시작_배송완료_category']=pd.cut(x=df_SMDW['배송시작_배송완료'],bins=bins,labels=labels)
# df_data[:10]
df_SMDW['배송시작_배송완료_category'].value_counts()

0-5      44228
6-10      2266
11-20      105
21-30       26
31 +         3
Name: 배송시작_배송완료_category, dtype: int64

In [10]:
bins = [0,5,10,20,30,100000]
labels = ['0-5','6-10','11-20','21-30','31 +']
df_SMDW['주문일_배송완료_category']=pd.cut(x=df_SMDW['주문일_배송완료'],bins=bins,labels=labels)
# df_data[:10]
df_SMDW['주문일_배송완료_category'].value_counts()

0-5      29921
6-10     15531
11-20     1173
21-30       80
31 +        30
Name: 주문일_배송완료_category, dtype: int64

In [11]:
bins = [-1,0,1,2,3,4,10,15,20,30,100000]
labels = ['0','1','2','3','4','5-10','11-15','15-20','21-30','31 +']
df_SMDW['주문일_배송시작_category']=pd.cut(x=df_SMDW['주문일_배송시작'],bins=bins,labels=labels)
# df_data[:10]
df_SMDW['주문일_배송시작_category'].value_counts()

1        17603
0        13917
2         7440
3         3966
4         2076
5-10      1548
11-15      119
15-20       40
21-30       25
31 +        15
Name: 주문일_배송시작_category, dtype: int64

In [12]:
df_SMDW

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송시작일_date,...,배송시작일_time,배송완료일_time,주문일_date,배송시작_배송완료,주문일_배송시작,주문일_배송완료,수량_category,배송시작_배송완료_category,주문일_배송완료_category,주문일_배송시작_category
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077.0,1,PC쇼핑몰,20181227,2019-01-07,...,2019-01-07 08:56:00,2019-01-11 05:32:00,2018-12-27,4,11,15,1,0-5,11-20,11-15
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42.0,2,모바일웹,20181229,2019-01-03,...,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-29,4,5,9,2-5,0-5,6-10,5-10
2,무통장입금,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,1271.0,1,모바일웹,20181230,2019-01-03,...,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-30,4,4,8,1,0-5,6-10,4
3,무통장입금,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,1085.0,1,모바일웹,20181231,2019-01-03,...,2019-01-03 09:30:00,2019-01-07 05:32:00,2018-12-31,4,3,7,1,0-5,6-10,3
4,무통장입금,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,1106.0,1,모바일웹,20181231,2019-01-07,...,2019-01-07 08:56:00,2019-01-11 05:32:00,2018-12-31,4,7,11,1,0-5,11-20,5-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46744,카드,D-20171229-0000036-00,2017-12-29 오후 4:32,2018-01-02 오전 5:31,52800,445.0,1,모바일웹,20171229,2017-12-29,...,2017-12-29 16:32:00,2018-01-02 05:31:00,2017-12-29,4,0,4,1,0-5,0-5,0
46745,카드,D-20171229-0000043-00,2018-01-03 오전 9:45,2018-01-07 오전 5:38,112200,172.0,2,모바일웹,20171229,2018-01-03,...,2018-01-03 09:45:00,2018-01-07 05:38:00,2017-12-29,4,5,9,2-5,0-5,6-10,5-10
46746,카드,D-20171230-0000022-01,2018-01-02 오후 7:06,2018-01-11 오전 2:46,36000,140.0,1,네이버 페이,20171230,2018-01-02,...,2018-01-02 19:06:00,2018-01-11 02:46:00,2017-12-30,9,3,12,1,6-10,11-20,3
46747,카드,D-20171231-0000017-01,2018-01-03 오후 2:00,2018-01-06 오후 6:06,73460,184.0,1,네이버 페이,20171231,2018-01-03,...,2018-01-03 14:00:00,2018-01-06 18:06:00,2017-12-31,3,3,6,1,0-5,6-10,3


In [13]:
df_SMDW['주문일_배송완료_category'].describe()

count     46735
unique        5
top         0-5
freq      29921
Name: 주문일_배송완료_category, dtype: object

In [14]:
df_SMDW['주문일_배송완료_category'].value_counts()

0-5      29921
6-10     15531
11-20     1173
21-30       80
31 +        30
Name: 주문일_배송완료_category, dtype: int64

In [15]:
df_SMDW['주문일_배송시작_category'].describe()

count     46749
unique       10
top           1
freq      17603
Name: 주문일_배송시작_category, dtype: object

In [16]:
df_SMDW['주문일_배송시작_category'].value_counts()

1        17603
0        13917
2         7440
3         3966
4         2076
5-10      1548
11-15      119
15-20       40
21-30       25
31 +        15
Name: 주문일_배송시작_category, dtype: int64

In [17]:
df_SMDW['배송시작_배송완료_category'].describe()

count     46628
unique        5
top         0-5
freq      44228
Name: 배송시작_배송완료_category, dtype: object

In [18]:
df_SMDW['배송시작_배송완료_category'].value_counts()

0-5      44228
6-10      2266
11-20      105
21-30       26
31 +         3
Name: 배송시작_배송완료_category, dtype: int64

### 기술 데이터 분석(DDA)
- 배송지연 기준을 주문일-배송시작의 간격이 5일 이상일 때, 배송시작-배송완료의 간격이 6일 이상일 때로 설정

In [19]:
condition_order_depart_on = "주문일_배송시작 >= 5"
condition_order_depart_off = "주문일_배송시작 < 5"
condition_depart_arrive_on = "배송시작_배송완료 >= 5"
condition_depart_arrive_off = "배송시작_배송완료 < 5"

#### 주문일-배송시작/결제수단
- 특정 결제 수단이 배송지연과 연관이 있는지 확인
- 결제수단 별 배송지연될 확률을 계산해본 결과  '페이나우(카드),무통장입금','예치금,휴대폰','적립금,카드,무통장입금','무통장입금,실시간계좌이체'가 높은 값이 나옴
- 하지만, 해당하는 수단들의 전체 데이터가 적어 결제수단과 배송지연을 연관짓기 어려움

In [20]:
# 주문일-배송시작이 5일 이상인 데이터 중 결제수단의 분포 확인
df_SMDW.query(condition_order_depart_on)['결제수단'].value_counts()

무통장입금             1149
카드                 286
적립금,무통장입금          111
쿠폰,무통장입금            56
쿠폰,카드               35
적립금,카드              24
카드,무통장입금            24
예치금,무통장입금            9
쿠폰,휴대폰               8
휴대폰                  8
실시간계좌이체              7
페이나우(카드)             6
예치금,적립금,무통장입금        5
예치금,휴대폰              4
무통장입금,휴대폰            3
카카오페이(카카오머니)         2
쿠폰,카드,무통장입금          2
가상계좌                 2
에스크로(실시간계좌이체)        2
페이나우(카드),무통장입금       1
무통장입금,실시간계좌이체        1
적립금,카드,무통장입금         1
페이나우(실시간 계좌이체)       1
Name: 결제수단, dtype: int64

In [21]:
# 결제수단 별, 주문일-배송시작이 5일 이상일 확률 계산
percent_order_depart_on_pay = (df_SMDW.query(condition_order_depart_on)['결제수단'].value_counts()/df_SMDW['결제수단'].value_counts())*100
percent_order_depart_on_pay.sort_values(ascending=False)

페이나우(카드),무통장입금        100.000000
예치금,휴대폰                80.000000
적립금,카드,무통장입금           50.000000
무통장입금,실시간계좌이체          50.000000
무통장입금,휴대폰              33.333333
카드,무통장입금               24.000000
쿠폰,카드,무통장입금            15.384615
페이나우(카드)               12.000000
쿠폰,휴대폰                 11.940299
예치금,적립금,무통장입금          10.638298
에스크로(실시간계좌이체)          10.000000
카카오페이(카카오머니)            9.090909
예치금,무통장입금               5.142857
페이나우(실시간 계좌이체)          4.761905
무통장입금                   4.753630
쿠폰,무통장입금                4.651163
적립금,무통장입금               3.289864
쿠폰,카드                   2.679939
실시간계좌이체                 2.173913
카드                      2.087134
적립금,카드                  1.880878
휴대폰                     1.860465
가상계좌                    1.538462
에스크로(가상계좌)                   NaN
예치금                          NaN
예치금,실시간계좌이체                  NaN
예치금,적립금,카드                   NaN
예치금,카드                       NaN
예치금,쿠폰,무통장입금                 NaN
예치금,쿠폰,카드                    NaN
적립금       

In [22]:
# 주문일-배송시작이 5일 미만인 데이터 중 결제수단의 분포 확인
df_SMDW.query(condition_order_depart_off)['결제수단'].value_counts()

무통장입금                 23022
카드                    13417
적립금,무통장입금              3263
쿠폰,카드                  1271
적립금,카드                 1252
쿠폰,무통장입금               1148
휴대폰                     422
실시간계좌이체                 315
예치금,무통장입금               166
가상계좌                    128
카카오페이(카드)                77
카드,무통장입금                 76
쿠폰,휴대폰                   59
페이나우(카드)                 44
예치금,적립금,무통장입금            42
적립금,휴대폰                  37
적립금,가상계좌                 30
예치금,카드                   28
예치금                      25
카카오페이(카카오머니)             20
페이나우(실시간 계좌이체)           20
에스크로(실시간계좌이체)            18
쿠폰,카카오페이(카드)             16
적립금                      16
쿠폰,페이나우(카드)              11
쿠폰,카드,무통장입금              11
적립금,페이나우(카드)              9
예치금,적립금,카드                8
적립금,카카오페이(카드)             7
적립금,실시간계좌이체               6
무통장입금,휴대폰                 6
쿠폰,에스크로(실시간계좌이체)          6
예치금,쿠폰,무통장입금              6
예치금,쿠폰,카드                 4
쿠폰,가상계좌                   4
적립금,카카오페이(카카오머니)    

In [23]:
# 결제수단 별, 주문일-배송시작이 5일 미만일 확률 계산
percent_order_depart_off_pay = (df_SMDW.query(condition_order_depart_off)['결제수단'].value_counts()/df_SMDW['결제수단'].value_counts())*100
percent_order_depart_off_pay.sort_values(ascending=False)

예치금,적립금,카드            100.000000
적립금,카카오페이(카드)         100.000000
적립금,페이나우(실시간 계좌이체)    100.000000
적립금,실시간계좌이체           100.000000
적립금,페이나우(카드)          100.000000
적립금,가상계좌              100.000000
적립금                   100.000000
적립금,휴대폰               100.000000
예치금,쿠폰,카드             100.000000
예치금,쿠폰,무통장입금          100.000000
예치금,카드                100.000000
적립금,카카오페이(카카오머니)      100.000000
카카오페이(카드)             100.000000
예치금,실시간계좌이체           100.000000
쿠폰,가상계좌               100.000000
예치금                   100.000000
쿠폰,에스크로(실시간계좌이체)      100.000000
에스크로(가상계좌)            100.000000
쿠폰,예치금                100.000000
쿠폰,카카오페이(카드)          100.000000
쿠폰,페이나우(카드)           100.000000
가상계좌                   98.461538
휴대폰                    98.139535
적립금,카드                 98.119122
카드                     97.912866
실시간계좌이체                97.826087
쿠폰,카드                  97.320061
적립금,무통장입금              96.710136
쿠폰,무통장입금               95.348837
무통장입금                  95.246370
페이나우(실시간 계

#### 주문일-배송시작/주문경로
- 특정 주문경로가 배송지연과 연관이 있는지 확인
- 특정 주문경로에서 배송지연이 발생한 경우를 발견하기 어려움

In [24]:
# 주문일-배송시작이 5일 이상인 데이터 중 주문경로의 분포 확인
df_SMDW.query(condition_order_depart_on)['주문경로'].value_counts()

PC쇼핑몰     1096
모바일웹       592
네이버 페이      59
Name: 주문경로, dtype: int64

In [25]:
# 주문경로 별, 주문일-배송시작이 5일 이상일 확률 계산
percent_order_depart_on_rout = (df_SMDW.query(condition_order_depart_on)['주문경로'].value_counts()/df_SMDW['주문경로'].value_counts())*100
percent_order_depart_on_rout.sort_values(ascending=False)

PC쇼핑몰     4.265421
모바일웹      3.194647
네이버 페이    2.338486
Name: 주문경로, dtype: float64

In [26]:
# 주문일-배송시작이 5일 미만인 데이터 중 주문경로의 분포 확인
df_SMDW.query(condition_order_depart_off)['주문경로'].value_counts()

PC쇼핑몰     24599
모바일웹      17939
네이버 페이     2464
Name: 주문경로, dtype: int64

In [27]:
# 주문경로 별, 주문일-배송시작이 5일 미만일 확률 계산
percent_order_depart_off_rout = (df_SMDW.query(condition_order_depart_off)['주문경로'].value_counts()/df_SMDW['주문경로'].value_counts())*100
percent_order_depart_off_rout.sort_values(ascending=False)

네이버 페이    97.661514
모바일웹      96.805353
PC쇼핑몰     95.734579
Name: 주문경로, dtype: float64

#### 주문일-배송시작/주문일
- 특정 주문일이 배송지연과 관련이 있는지 확인
- 명절, 기념일에 주문 시, 배송 지연의 확률이 높았음
- 수량과 주문일의 상관관계를 EDA에서 확인 필요

In [28]:
# 주문일-배송시작이 5일 이상인 데이터 중 주문일 분포 확인
df_SMDW.query(condition_order_depart_on)['주문일_date'].value_counts().iloc[:10]

2019-02-27    68
2019-02-28    66
2018-10-05    57
2019-01-30    43
2018-05-18    38
2018-12-21    37
2019-01-31    35
2019-05-03    34
2018-05-11    29
2018-09-21    28
Name: 주문일_date, dtype: int64

In [29]:
# 주문일 별, 주문일-배송시작이 5일 이상일 확률 계산
percent_order_depart_on_order_date = (df_SMDW.query(condition_order_depart_on)['주문일_date'].value_counts()/df_SMDW['주문일_date'].value_counts())*100
percent_order_depart_on_order_date.sort_values(ascending=False).iloc[:10]

2017-01-01    100.0
2017-05-13    100.0
2019-06-02    100.0
2019-02-03    100.0
2019-02-02    100.0
2019-02-01    100.0
2019-01-31    100.0
2018-09-21    100.0
2017-10-03    100.0
2017-09-29    100.0
Name: 주문일_date, dtype: float64

In [30]:
# 주문일-배송시작이 5일 미만인 데이터 중 주문일의 분포 확인
df_SMDW.query(condition_order_depart_off)['주문일_date'].value_counts()

2019-01-28    329
2018-12-26    249
2019-04-03    249
2018-11-08    242
2018-11-19    239
             ... 
2017-10-22      1
2017-08-05      1
2017-03-25      1
2017-03-24      1
2017-09-28      1
Name: 주문일_date, Length: 792, dtype: int64

In [31]:
# 주문일 별, 주문일-배송시작이 5일 미만일 확률 계산
percent_order_depart_off_order_date = (df_SMDW.query(condition_order_depart_off)['주문일_date'].value_counts()/df_SMDW['주문일_date'].value_counts())*100
percent_order_depart_off_order_date.sort_values(ascending=False)

2017-01-15    100.0
2018-03-09    100.0
2018-03-28    100.0
2018-03-27    100.0
2018-03-26    100.0
              ...  
2019-02-01      NaN
2019-02-02      NaN
2019-02-03      NaN
2019-06-02      NaN
2019-06-04      NaN
Name: 주문일_date, Length: 807, dtype: float64

#### 배송시작-배송완료/상품번호
- 특정 상품과 배송지연이 연관되어있는지 확인
- 100% 배송 지연된 상품의 경우 상품이 1개여서 상품번호와 배송 지연을 연관짓기에는 어려움이 있음
- 상품번호와 연관 짓기 위해서는 수량을 같이 비교 고려

In [32]:
# 배송시작-배송완료이 5일 이상인 데이터 중 상품번호 분포 확인
df_SMDW.query(condition_order_depart_on)['상품번호'].value_counts().iloc[:10]

455.0     51
608.0     44
1374.0    42
1373.0    42
1059.0    34
1237.0    34
1077.0    33
1238.0    31
172.0     31
1036.0    30
Name: 상품번호, dtype: int64

In [33]:
# 상품번호 별, 배송시작-배송완료 5일 이상일 확률 계산
percent_depart_arrive_on_product_num= (df_SMDW.query(condition_order_depart_on)['상품번호'].value_counts()/df_SMDW['상품번호'].value_counts())*100
percent_depart_arrive_on_product_num.sort_values(ascending=False).iloc[:10]

1625.0    100.0
1526.0    100.0
1022.0    100.0
156.0     100.0
143.0     100.0
1158.0    100.0
1192.0    100.0
1608.0    100.0
1556.0    100.0
1502.0     50.0
Name: 상품번호, dtype: float64

In [34]:
# 배송시작-배송완료이 5일 미만인 데이터 중 상품번호의 분포 확인
df_SMDW.query(condition_order_depart_off)['상품번호'].value_counts()

455.0     1649
1237.0    1543
1374.0    1444
1209.0    1428
1373.0    1411
          ... 
1208.0       1
1241.0       1
1541.0       1
1245.0       1
333.0        1
Name: 상품번호, Length: 639, dtype: int64

In [35]:
# 상품번호 별, 배송시작-배송완료이 5일 미만일 확률 계산
percent_depart_arrive_off_product_num = (df_SMDW.query(condition_order_depart_off)['상품번호'].value_counts()/df_SMDW['상품번호'].value_counts())*100
percent_depart_arrive_off_product_num.sort_values(ascending=False)

942.0     100.0
1044.0    100.0
1124.0    100.0
1123.0    100.0
1121.0    100.0
          ...  
1192.0      NaN
1526.0      NaN
1556.0      NaN
1608.0      NaN
1625.0      NaN
Name: 상품번호, Length: 648, dtype: float64